#Обработка данных

###Импорт библиотек и данных

> Добавить блок с цитатой



In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [70]:
history=pd.read_csv('https://drive.google.com/uc?export=download&id=1SmFlAHKnaqPy-lp8H3ZPegqQpz6C6gUq', sep='\t')
users=pd.read_csv('https://drive.google.com/uc?export=download&id=16UVDTOX6Gli4amZnDsRrow11ywI-WHl2', sep='\t')
validate_answers=pd.read_csv('https://drive.google.com/uc?export=download&id=1Nz-z3heM8zhUYahcpcsQ7wXxYUo-Ud2B', sep='\t')
validate=pd.read_csv('https://drive.google.com/uc?export=download&id=121sJ7OJ-ZXKAElaXiNHNsuZ49Qmbd2ki', sep='\t')

###Обработка выбросов

In [71]:
users = users[(users['age'] >= 6) & (users['age'] <= 90)]
users = users[users['sex'] != 0]

###Объединение датасетов + новые признаки

In [72]:
history['hour_mod_24'] = history['hour'] % 24

In [74]:
def create_users_parametrs(user_ids):
    user_ = users[users['user_id'].isin(user_ids)]

    if user_.empty:
        return pd.Series({
            'average_age': 0,
            'percent_6_15': 0,
            'percent_16_25': 0,
            'percent_26_45': 0,
            'percent_46_60': 0,
            'percent_60_75': 0,
            'percent_70_90': 0,
            'percent_men': 0,
            'percent_women': 0,
            'unique_cities': 0,
            'percent_cities_3_7_19_25': 0
        })

    average_age = user_['age'].mean()
    total_users = len(user_)
    percent_6_15 = (user_[(user_['age'] >= 6) & (user_['age'] <= 15)].shape[0] / total_users) * 100
    percent_16_25 = (user_[(user_['age'] >= 16) & (user_['age'] <= 25)].shape[0] / total_users) * 100
    percent_26_45 = (user_[(user_['age'] >= 26) & (user_['age'] <= 45)].shape[0] / total_users) * 100
    percent_46_60 = (user_[(user_['age'] >= 46) & (user_['age'] <= 60)].shape[0] / total_users) * 100
    percent_60_75 = (user_[(user_['age'] >= 60) & (user_['age'] <= 75)].shape[0] / total_users) * 100
    percent_70_90 = (user_[(user_['age'] >= 70) & (user_['age'] <= 90)].shape[0] / total_users) * 100

    percent_men = (user_[user_['sex'] == 1].shape[0] / total_users) * 100
    percent_women = (user_[user_['sex'] == 2].shape[0] / total_users) * 100

    unique_cities = user_['city_id'].nunique()
    cities_of_interest = [3, 7, 19, 25]
    percent_big_cities = (user_[user_['city_id'].isin(cities_of_interest)].shape[0] / total_users) * 100

    return pd.Series({
        'average_age': average_age,
        'percent_6_15': percent_6_15,
        'percent_16_25': percent_16_25,
        'percent_26_45': percent_26_45,
        'percent_46_60': percent_46_60,
        'percent_60_75': percent_60_75,
        'percent_70_90': percent_70_90,
        'percent_men': percent_men,
        'percent_women': percent_women,
        'unique_cities': unique_cities,
        'percent_cities_3_7_19_25': percent_big_cities
    })

In [75]:
result = validate.merge(validate_answers, left_index=True, right_index=True)
result['user_ids'] = result['user_ids'].apply(lambda x: eval(x))
params = result['user_ids'].apply(create_users_parametrs)
result = pd.concat([result, params], axis=1)

In [76]:
def create_publishers_parametrs(publishers):
    if isinstance(publishers, str):
        publishers = eval(publishers)
    elif isinstance(publishers, int):
        publishers = [publishers]

    filtered_history = history[history['publisher'].isin(publishers)]

    if filtered_history.empty:
        return pd.Series({
            'average_cpm': 0,
            'min_cpm': 0,
            'max_cpm': 0,
            'most_active_hour': None,
            'second_active_hour': None,
            'third_active_hour': None,
            'fourth_active_hour': None,
            'fifth_active_hour': None
        })

    average_cpm = filtered_history['cpm'].mean()
    min_cpm = filtered_history['cpm'].min()
    max_cpm = filtered_history['cpm'].max()

    active_hours = filtered_history['hour_mod_24'].value_counts().nlargest(5)

    most_active_hour = active_hours.index[0] if len(active_hours) > 0 else None
    second_active_hour = active_hours.index[1] if len(active_hours) > 1 else None
    third_active_hour = active_hours.index[2] if len(active_hours) > 2 else None
    fourth_active_hour = active_hours.index[3] if len(active_hours) > 3 else None
    fifth_active_hour = active_hours.index[4] if len(active_hours) > 4 else None

    return pd.Series({
        'average_cpm': average_cpm,
        'min_cpm': min_cpm,
        'max_cpm': max_cpm,
        'most_active_hour': most_active_hour,
        'second_active_hour': second_active_hour,
        'third_active_hour': third_active_hour,
        'fourth_active_hour': fourth_active_hour,
        'fifth_active_hour': fifth_active_hour
    })

In [78]:
result['publishers'] = result['publishers'].apply(lambda x: eval(x))
cpm_metrics = result['publishers'].apply(create_publishers_parametrs)
result = pd.concat([result, cpm_metrics], axis=1)

In [79]:
result['duration'] = result['hour_end'] - result['hour_start']